In [255]:
!pip install xgboost
!pip install lightgbm

In [256]:
import pandas as pd
import numpy as np
from sklearn import model_selection

from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix   #gerekebilir diye ekledim

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

import xgboost
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from sklearn.exceptions import ConvergenceWarning

In [257]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)  
warnings.filterwarnings("ignore", category=ConvergenceWarning) 

# Çağırdığımız Fonksiyonumuz
## (İşlemTürü , Verimiz , HedefColumn , Eğitim Yüzdesi)

In [261]:
def the_function(process_type,df,target_param,training_per):

    if process_type==1: #classification
        X=df.drop([target_param],axis=1)      #Bağımsız Değişken
        y=df[target_param]                    #Bağımlı Değişken
        X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=training_per,random_state=123,shuffle=True)
        # Modellerin listesinin olusturulmasi
        models = [
            ("LR", LogisticRegression(max_iter=10000,solver = "liblinear")),
            ("LDA", LinearDiscriminantAnalysis()),
            ("KNN", KNeighborsClassifier()),
            ("DT", DecisionTreeClassifier()),
            ("NB", GaussianNB()),
            ("SVM", SVC()),
            ("RF", RandomForestClassifier(max_features = 8, 
                                      min_samples_split = 5, 
                                      n_estimators = 500)),
        ]
        sonuc = []
        sonuclar = pd.DataFrame(columns= ["Modeller","Accuracy","Precision","Recall","AUC"])
        # Modeller için 'cross validation' sonuçlarının  yazdırılması
        for name, model in models:
            current_model = model.fit(X_train, y_train)
            y_pred = current_model.predict(X_test)
            
            accuracy = accuracy_score(y_test, y_pred)
            precision=precision_score(y_test, y_pred, average='weighted')  #micro macro weighted olarak farklı parametrelerle çalışır
            recall=recall_score(y_test, y_pred, average='weighted')        #micro macro weighted olarak farklı parametrelerle çalışır
            auc=roc_auc_score(y_test, y_pred)
            sonuc = pd.DataFrame([[name, accuracy,precision,recall,auc]], columns= ["Modeller","Accuracy","Precision","Recall","AUC"])
            sonuclar = sonuclar.append(sonuc)
        return sonuclar
        
        
    if process_type==2: #regression
        #train-test ayrimi
        y = df[target_param]
        X=df.drop([target_param],axis=1)
        X_train,X_test,y_train,y_test=train_test_split(X,y,train_size=training_per,random_state=123,shuffle=True)
        #modelleme
        
        models = [
            ("LGBM", LGBMRegressor()),
            ("XGBR", XGBRegressor()),
            ("GBR", GradientBoostingRegressor()),
            ("RFR", RandomForestRegressor(random_state = 42)),
            ("DT", DecisionTreeRegressor(max_leaf_nodes = 10)),
            ("MLP", MLPRegressor()),
            ("Knn", KNeighborsRegressor()),
            ("SVR", SVR("linear")),
        ]
        sonuc = []
        sonuclar = pd.DataFrame(columns= ["Modeller","MAE","RMSE","MAPE","R-Sqrt"])
        for name,model in models:
            current_model = model.fit(X_train, y_train)                 #Model Oluştur
            y_pred = model.predict(X_test)                               #Test ile predict ettik predict edilen sonuçları 
                                                                         #gerçek sonuçlarla kıyaslıcaz aşağıda farklı 4 metrikle
            MSE=median_absolute_error(y_test, y_pred)                    #MSE
            RMSE = np.sqrt(mean_squared_error(y_test, y_pred))           #RMSE
            y_true, y_predi = np.array(y_test), np.array(y_pred)
            MAPE=np.mean(np.abs((y_true - y_predi) / y_true)) * 100      #MAPE Hesaplama
            Rsqrt=r2_score(y_test, y_pred)                               #R-SQRT Hesaplama
        
            sonuc = pd.DataFrame([[name,MSE, RMSE,MAPE,Rsqrt]], columns= ["Modeller","MAE","RMSE","MAPE","R-Sqrt"])
            sonuclar = sonuclar.append(sonuc)
        return sonuclar
    print("DONE!!!")
    


# Sınıflandırma için kullanacaksak: Bu şekilde çağırırız(processtype=1)

In [262]:
df = pd.read_csv("diabetes.csv")  #Target= "Outcome"  #Classification
the_function(1,df,"Outcome",0.2)

,Modeller,Accuracy,Precision,Recall,AUC
0,LR,0.715447,0.706963,0.715447,0.666628
0,LDA,0.759350,0.753799,0.759350,0.718011
0,KNN,0.705691,0.701133,0.705691,0.665890
0,DT,0.712195,0.720531,0.712195,0.694327
0,NB,0.757724,0.754616,0.757724,0.724595
0,SVM,0.754472,0.749097,0.754472,0.687462
0,RF,0.773984,0.770470,0.773984,0.740355


# Regresyon için kullanacaksak: Bu şekilde çağırırız(processtype=2)

In [263]:
df = pd.read_csv("Hitters.csv")   #Target="Salary"   #Regression
df = df.dropna()
df = df.drop(['League', 'Division', 'NewLeague'], axis=1).astype('float64')
the_function(2,df,"Salary",0.2)


,Modeller,MAE,RMSE,MAPE,R-Sqrt
0,LGBM,191.027763,340.967846,76.368084,0.325573
0,XGBR,79.876022,302.864973,41.758571,0.467884
0,GBR,90.049031,295.655242,43.439930,0.492917
0,RFR,80.441650,315.514742,45.651920,0.422506
0,DT,116.128231,346.500760,58.125648,0.303507
0,MLP,142.599487,431.708280,79.220874,-0.081157
0,Knn,99.000000,378.612858,53.729908,0.168430
0,SVR,180.261434,408.512558,73.255309,0.031903
